In [4]:
import json
import networkx as nx
from networkx.algorithms.simple_paths import shortest_simple_paths

# Load the JSON with precomputed paths (no domestic hops)
with open('no_domestic_hops.json') as f:
    data = json.load(f)

# Build the graph from features
graph = nx.Graph()
node_coords = {}

for feature in data['features']:
    path = feature['properties']['path']
    cables = feature['properties']['cables']
    for node in path:
        graph.add_node(node)
    for u, v, cable in zip(path, path[1:], cables):
        graph.add_edge(u, v, cable=cable)

# Collect unique cables
all_cables = sorted({d['cable'] for _, _, d in graph.edges(data=True)})

def list_routes(G, source, target, k=5):
    try:
        paths = shortest_simple_paths(G, source, target)
        results = []
        for _, p in zip(range(k), paths):
            cables = [G.get_edge_data(p[i], p[i+1])['cable'] for i in range(len(p)-1)]
            results.append((p, cables))
        return results
    except nx.NetworkXNoPath:
        return []

if __name__ == '__main__':
    # Display available cables with numbering
    print("Available cables to disable:")
    for idx, cable in enumerate(all_cables, 1):
        print(f" {idx}. {cable}")

    # Prompt user to select cable
    choice = input("\nEnter the number of the cable to disable (e.g. 1): ")
    try:
        choice_index = int(choice) - 1
        disabled_cable = all_cables[choice_index]
    except (ValueError, IndexError):
        print(f"Invalid selection '{choice}'. Exiting.")
        exit(1)

    # Remove the chosen cable's edges
    H = graph.copy()
    to_remove = [(u, v) for u, v, d in H.edges(data=True) if d.get('cable') == disabled_cable]
    H.remove_edges_from(to_remove)
    print(f"Removed edges for cable '{disabled_cable}' (total removed: {len(to_remove)})\n")

    # Prompt for source and target
    source = input("Enter source node (e.g. Trivandrum, India): ")
    target = input("Enter target node (e.g. Brisbane, Australia): ")

    print(f"\nFinding alternate routes from {source} to {target}...\n")
    alt_routes = list_routes(H, source, target, k=5)
    if not alt_routes:
        print("No alternate path found.")
    else:
        for idx, (path, cables) in enumerate(alt_routes, 1):
            print(f"Route {idx}:")
            print("  Path:  ", " -> ".join(path))
            print("  Cables:", " -> ".join(cables), "\n")


Available cables to disable:
 1. Apricot
 2. Asia Africa Europe-1 (AAE-1)
 3. Asia Connect Cable-1 (ACC-1)
 4. Asia-America Gateway (AAG) Cable System
 5. Australia-Singapore Cable (ASC)
 6. Batam Dumai Melaka (BDM)
 7. Bay of Bengal Gateway (BBG)
 8. Bharat Lanka Cable System
 9. Bifrost
 10. Coral Sea Cable System (CS²)
 11. FALCON
 12. Hawaiki Nui 1
 13. ICE IV
 14. INDIGO-West
 15. India Asia Xpress (IAX)
 16. Indonesia Global Gateway (IGG) System
 17. JAKABARE
 18. Jakarta-Bangka-Bintan-Batam-Singapore (B3JS)
 19. MIST
 20. SAFE
 21. SEA-H2X
 22. SeaMeWe-4
 23. SeaMeWe-5
 24. Tata TGN-Tata Indicom
 25. Thailand-Indonesia-Singapore (TIS)


Removed edges for cable 'Tata TGN-Tata Indicom' (total removed: 1)


Finding alternate routes from Chennai, India to Brisbane, Australia...

Route 1:
  Path:   Chennai, India -> Jakarta, Indonesia -> Brisbane, Australia
  Cables: ICE IV -> Hawaiki Nui 1 

Route 2:
  Path:   Chennai, India -> Singapore, Singapore -> Brisbane, Australia
  Cables: ICE IV -> Hawaiki Nui 1 

Route 3:
  Path:   Chennai, India -> Ancol, Indonesia -> Brisbane, Australia
  Cables: ICE IV -> Hawaiki Nui 1 

Route 4:
  Path:   Chennai, India -> Jakarta, Indonesia -> Singapore, Singapore -> Brisbane, Australia
  Cables: ICE IV -> Hawaiki Nui 1 -> Hawaiki Nui 1 

Route 5:
  Path:   Chennai, India -> Singapore, Singapore -> Jakarta, Indonesia -> Brisbane, Australia
  Cables: ICE IV -> Hawaiki Nui 1 -> Hawaiki Nui 1 

